In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import copy

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

filelist = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.endswith(".in"):
            filelist.append(os.path.join(dirname, filename))

for filename in filelist:
    
    print('Process file: ' + filename)

    # Part 0: Reading Input File
    with open(filename, 'r') as data:
        first_line = data.readline()
        rows, cols, num_drones, num_turns, payload = first_line.split()
        rows, cols, num_drones, num_turns, payload = int(rows), int(cols), int(num_drones), int(num_turns), int(payload)
    
        second_line = data.readline()
        num_prodtypes = int(second_line)
        
        third_line = data.readline()
        prodtype_weights = [int(x) for x in third_line.split()]
        
        fourth_line = data.readline()
        num_warehouses = int(fourth_line)
        
        # 2D Array, row = warehouse, col = (x, y) coordinates
        warehouse_locs = []
        # 2D Array, row = warehouse, col = quantity of each prod
        warehouse_prods = []
        for i in range(num_warehouses):
            wh_coords = data.readline()
            warehouse_locs.append([int(x) for x in wh_coords.split()])
            prod_qty = data.readline()
            warehouse_prods.append([int(x) for x in prod_qty.split()])
        
        num_orders = data.readline()
        num_orders = int(num_orders)
        
        # 2D Array, row = order, col = (x, y) coordinates
        order_locs = []
        # 2D Array, row = warehouse, col = quantity of each prod type ordered
        order_prods = []
        for i in range(num_orders):
            order_coords = data.readline()
            order_locs.append([int(x) for x in order_coords.split()])
            num_prods_ordered = data.readline()
            num_prods_ordered = int(num_prods_ordered)
            
            prods_ordered = [0] * num_prodtypes
                        
            prodtype_ordered = data.readline()
            prodtype_ordered = [int(x) for x in prodtype_ordered.split()]
            
            for prodtype in prodtype_ordered:
                prods_ordered[prodtype] += 1
            
            order_prods.append(prods_ordered)
        
        prodtype_weights = np.array(prodtype_weights)
        warehouse_locs = np.array(warehouse_locs)
        warehouse_prods = np.array(warehouse_prods)
        order_locs = np.array(order_locs)
        order_prods = np.array(order_prods)
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Distance between orders
order_locs_reshape = order_locs.reshape(order_locs.shape[0], 1, order_locs.shape[1])
order_dist_matrix = np.sqrt(np.einsum('ijk, ijk->ij', order_locs - order_locs_reshape, order_locs - order_locs_reshape))
longest_order_dist = np.max(order_dist_matrix)

# Cij for local search
coord_matrix = np.vstack((warehouse_locs, order_locs))
coord_matrix_reshape = coord_matrix.reshape(coord_matrix.shape[0], 1, coord_matrix.shape[1])
c_ij = np.sqrt(np.einsum('ijk, ijk->ij', coord_matrix - coord_matrix_reshape, coord_matrix - coord_matrix_reshape))

In [ ]:
# 7:13 pm
# functions

# drone_route -> merged_route
# for the former, it is actually the command for drones
# for the latter, the location index for orders +10 to fit the distance matrix

def dr2mr(r):
    r = r[:]
    r.append(['x','x','x','x','x'])
    sol = []
    loc = -1
    temp_list = []
    prod_list = []
    for i in range(len(r)):
        if r[i][2] != loc or r[i][1] != r[i-1][1]:
            if loc != -1:
                sol.append(copy.deepcopy(temp_list))
                temp_list.clear()
            prod_list.clear()
            prod_list.append((r[i][3],r[i][4]))
            temp_list.append(r[i][0])
            temp_list.append(r[i][1])
            if r[i][1] == 'D':
                temp_list.append(r[i][2]+10)     # for orders +10
            else:
                temp_list.append(r[i][2])    
            temp_list.append(prod_list)
           
            loc = r[i][2]
        else:
            prod_list.append((r[i][3],r[i][4]))
        
    return sol



# merged_route -> drone_route
def mr2dr(r):
    sol = []
    for i in range(len(r)):
        for j in range(len(r[i][3])):
            if r[i][1] == 'L': 
                sol.append([r[i][0], r[i][1],r[i][2],r[i][3][j][0],r[i][3][j][1]])
            else:
                sol.append([r[i][0], r[i][1],r[i][2]-10,r[i][3][j][0],r[i][3][j][1]])
    return sol

# calculate route total distance
def getroute_length(r):
    l = 0
    
    for i in range(len(r) - 1):
        l += c_ij[r[i][2]][r[i+1][2]]
    return l


# 1-opt move and local search
# check whether index i can be inserted after index j
def is_insert_feasible(r,i,j):
    dic={}
    if r[i][1]=='D':
        for m in range(len(r[i][3])):
            dic[r[i][3][m][0]]=r[i][3][m][1]*(-1)
        for m in range(j+1):
            for n in range(len(r[m][3])):
                if  r[m][3][n][0] in dic.keys():
                    if r[m][1]=='L':
                        dic[r[m][3][n][0]]+=r[m][3][n][1]
                    else:
                        dic[r[m][3][n][0]]-=r[m][3][n][1]
        if min(dic.values())<0:
            return False
    else:
        for m in range(len(r[i][3])):
            dic[r[i][3][m][0]]=0
        for m in range(j+1):
            if m==i:
                continue
            for n in range(len(r[m][3])):
                if  r[m][3][n][0] in dic.keys():
                    if r[m][1]=='L':
                        dic[r[m][3][n][0]]+=r[m][3][n][1]
                    else:
                        dic[r[m][3][n][0]]-=r[m][3][n][1]
        if min(dic.values())<0:
            return False
    return True       

# calculate the delta distance
def delta_dis(r, i, j):
    if i == 0:
        if j != len(r) - 1:
            return - c_ij[r[i][2]][r[i+1][2]] - c_ij[r[j][2]][r[j+1][2]] + c_ij[r[j][2]][r[i][2]] + c_ij[r[i][2]][r[j+1][2]]
        else:
            return - c_ij[r[i][2]][r[i+1][2]] + c_ij[r[i][2]][r[j][2]]
    elif i != len(r)-1:
        if j != len(r) - 1:
            return - c_ij[r[i-1][2]][r[i][2]] - c_ij[r[i][2]][r[i+1][2]] - c_ij[r[j][2]][r[j+1][2]] + c_ij[r[i-1][2]][r[i+1][2]] + c_ij[r[i][2]][r[j][2]] + c_ij[r[i][2]][r[j+1][2]]
        else:
            return - c_ij[r[i-1][2]][r[i][2]] - c_ij[r[i][2]][r[i+1][2]] + c_ij[r[i][2]][r[j][2]]
    else:
        return - c_ij[r[i-1][2]][r[i][2]] - c_ij[r[j][2]][r[j+1][2]] + c_ij[r[i][2]][r[j][2]] + c_ij[r[j+1][2]][r[i][2]]


# local search by 1-opt with only downhill (better solution)
# time_0_flag = 0 means at time turn 0, do not move location 0

def local_search_1opt(r, time_0_flag = 0):
    for i in range(len(r)):
        for j in range(len(r)):
            if i == j or (time_0_flag == 0 and i == 0):
                continue
            else:
                if is_insert_feasible(r, i, j):
                    if delta_dis(r,i,j) < 0:
                        temp = r[i]
                        del r[i]
                        if i < j:
                            r.insert(j,temp)
                        else:
                            r.insert(j+1,temp)
    return None
  



def local_search_1opt(r, time_0_flag = 0):
    for i in range(len(r)):
        for j in range(len(r)):
            if i == j or (time_0_flag == 0 and i == 0):
                continue
            else:
                if is_insert_feasible(r, i, j):
                    if delta_dis(r,i,j) < 0:
                        temp = r[i]
                        del r[i]
                        if i < j:
                            r.insert(j,temp)
                        else:
                            r.insert(j+1,temp)
    return None

def r2r(r, time_0_flag = 0):
    BKS = copy.deepcopy(r)
    BKS_value = getroute_length(BKS)
    
    records = BKS_value * 0.01
    
    
    for i in range(len(r)):
        for j in range(len(r)):
            if i == j or (time_0_flag == 0 and i == 0):
                continue
            else:
                if is_insert_feasible(r, i, j):
                    if delta_dis(r,i,j) < 0:
                        temp = r[i]
                        del r[i]
                        if i < j:
                            r.insert(j,temp)
                        else:
                            r.insert(j+1,temp)
                        BKS = r
                        BKS_value = getroute_length(BKS)
                        records = BKS_value * 0.05
                    elif delta_dis(r,i,j) < records:
                        temp = r[i]
                        del r[i]
                        if i < j:
                            r.insert(j,temp)
                        else:
                            r.insert(j+1,temp)
    r = BKS
    return None
    
def local_search(r, time_0_flag = 0):
    BKS_Value = getroute_length(r)
    while 1:
        r2r(r)
        local_search_1opt(r)
        current = getroute_length(r)
        delta = BKS_Value - current
        if delta > 0:
            BKS_Value = current
        
        if delta <= 1e-5:
            break
    return None


###################################################  
def recalcDroneLocTime(partialSoln, orig_avail_time, orig_drone_loc, warehouse_locs, order_locs):
    
    avail_time = orig_avail_time
    drone_loc = orig_drone_loc
    
    for command in partialSoln:
        drone_id = command[0]
        if (command[1] == 'L') or (command[1] == 'U'):
            wh_id = command[2]
            curr_wh_loc = warehouse_locs[wh_id]
            dist_wh = np.linalg.norm(curr_wh_loc - drone_loc, 2)
            avail_time += (np.ceil(dist_wh) + 1)
            drone_loc = warehouse_locs[wh_id]
        elif command[1] == 'D':
            order_id = command[2]
            curr_order_loc = order_locs[order_id]
            dist_order = np.linalg.norm(curr_order_loc - drone_loc, 2)
            avail_time += (np.ceil(dist_order) + 1)
            drone_loc = order_locs[order_id]
    
    return avail_time, drone_loc

def partialSolnStrOutput(list_commands):    
    convertedPartialSoln = []
    for command in list_commands:
        if command[1] == 'W':
            convertedPartialSoln.append(str(command[0]) + " W " + str(command[2]))
        else:
            convertedPartialSoln.append(str(command[0]) + " " + str(command[1]) + " " + str(command[2]) + " " + str(command[3]) + " " + str(command[4]))
    return convertedPartialSoln

def calcPercFulfilled(deliver_prods, order_prods, prodtype_weights):
    return np.sum(deliver_prods * prodtype_weights) / np.sum(np.sum(order_prods * prodtype_weights))

In [ ]:
solution = [] # List of string

# Drone's timing
drone_avail_time = np.array([0] * num_drones)
drone_curr_loc = np.tile(warehouse_locs[0], (num_drones, 1))

mean_order_loc = np.mean(order_locs, axis = 0)

skip_counter = 0
# Any drone still available before total number of turns
while (np.any(drone_avail_time < num_turns) and (np.sum(order_prods) > 0) and (skip_counter < len(drone_avail_time))):
    
    # Choose the earliest available drone, skipping those without solutions
    drone_no = np.argsort(drone_avail_time)[skip_counter]
    # No drone to plan anymore
    if drone_avail_time[drone_no] >= num_turns:
        break
    
    print("Planning for drone " + str(drone_no) + " available at time " + str(drone_avail_time[drone_no]))
    
    # Item Planning Phase    
    avail_time = drone_avail_time[drone_no]
    drone_loc_plan = drone_curr_loc[drone_no]
    # Record available space in the drone
    avail_space = payload
    # Current drone solution
    drone_solution = []
    
    order_prods_plan = np.copy(order_prods)
    warehouse_prods_plan = np.copy(warehouse_prods)

    # Aggregate weights per order
    order_weights = order_prods_plan * prodtype_weights # 2D matrix, row = order, col = prodtype
    weights_per_order = np.sum(order_weights, axis = 1) # 1D vector, length = number of orders
    
    #dist_drone_order = np.linalg.norm(order_locs - drone_loc_plan, 2, axis = 1) # 1D vector, length = number of orders    
    #dist_mean_order = np.linalg.norm(order_locs - mean_order_loc, 2, axis = 1)    
    #score_per_order = 2/3 * weights_per_order / np.max(weights_per_order) + 1/3 * dist_drone_order / np.max(dist_drone_order)
    
    load_items = dict() # key = prodtype, val = units
    deliver_items = dict() # key = orderNo, val = load_dict
    
    order_prods_unitweights = (order_prods_plan > 0) * prodtype_weights
    min_item_weight = np.min(order_prods_unitweights[order_prods_unitweights > 0])
    while (avail_space > min_item_weight):
        
        #nonzero_order_idx, nonzero_order_idx = np.where(order_prods > 0)
        
        # Loading plan for all orders, 2D matrix
        all_orders_load_plan = []
        # Find out per-order load plan if drone is used to fulfill respective order
        for orderNo in range(order_prods_plan.shape[0]):

            avail_space_plan = avail_space
            
            curr_order_prods = order_prods_plan[orderNo]            
            curr_order_prodtypes = np.where(curr_order_prods > 0)[0]
            curr_order_prod_unitweights = prodtype_weights[curr_order_prodtypes]

            # Loading plan for this current order, 1D matrix
            curr_order_load_plan = np.array([0] * num_prodtypes)
            
            # Calculate percentage of order        
            # Heaviest item first
            for idx in curr_order_prod_unitweights.argsort()[::-1]:
                
                # If available space is less than the unit weight of this product
                if avail_space_plan < curr_order_prod_unitweights[idx]:
                    continue
                
                units_to_load = avail_space_plan // curr_order_prod_unitweights[idx]
                
                if units_to_load > order_prods_plan[orderNo, curr_order_prodtypes[idx]]:
                    units_to_load = order_prods_plan[orderNo, curr_order_prodtypes[idx]]
                
                curr_order_load_plan[curr_order_prodtypes[idx]] += units_to_load
                
                avail_space_plan -= (units_to_load * curr_order_prod_unitweights[idx])
            
            all_orders_load_plan.append(curr_order_load_plan)
        
        all_orders_load_plan = np.array(all_orders_load_plan)        

        all_orders_load_plan_weight = all_orders_load_plan * prodtype_weights        
        all_orders_load_plan_weight = np.sum(all_orders_load_plan_weight, axis = 1) # 1-D matrix, row = order

        order_prods_plan_weight = order_prods_plan * prodtype_weights
        order_prods_plan_weight = np.sum(order_prods_plan_weight, axis = 1)
        
        score_per_order = [float('-inf')] * num_orders # 1-D matrix, score for each order
        update_order_nos = np.where((order_prods_plan_weight > 0) & (all_orders_load_plan_weight > 0))[0]
        for orderNo in update_order_nos:
            score_per_order[orderNo] = all_orders_load_plan_weight[orderNo] / order_prods_plan_weight[orderNo]
            # Add order distance here, no weighting yet
            if len(deliver_items) > 0:
                score_per_order[orderNo] -= (np.mean(order_dist_matrix[orderNo, list(deliver_items.keys())]) / longest_order_dist)
        
        score_per_order = np.array(score_per_order)
        
        # Phase 1: Order and Product Assignment
        assigned_order = np.argsort(score_per_order)[-1]
        best_load_plan = all_orders_load_plan[assigned_order]
        
        # Continue here
        for load_prodtype in np.where(best_load_plan > 0)[0]:
            if load_prodtype in load_items:
                load_items[load_prodtype] += best_load_plan[load_prodtype]
            else:
                load_items[load_prodtype] = best_load_plan[load_prodtype]
            
            if assigned_order in deliver_items:
                if load_prodtype in deliver_items[assigned_order]:
                    deliver_items[assigned_order][load_prodtype] += best_load_plan[load_prodtype]
                else:
                    deliver_items[assigned_order][load_prodtype] = best_load_plan[load_prodtype]
            else:
                deliver_items[assigned_order] = {load_prodtype: best_load_plan[load_prodtype]}
        
        # Update min_item_weight and avail_space
        avail_space -= np.sum(best_load_plan * prodtype_weights)
        # order_prods_plan
        order_prods_plan[assigned_order] -= best_load_plan
        # update min_item_weight
        if np.sum(order_prods_plan) == 0: # All items assigned
            break
        else:
            order_prods_unitweights = (order_prods_plan > 0) * prodtype_weights
            min_item_weight = np.min(order_prods_unitweights[order_prods_unitweights > 0])
        
        #print(np.where(order_prods_unitweights[order_prods_unitweights > 0] == 2))        
    
    # Phase 2: Warehouse & Customer delivery route planning
    # Warehouse route planning
    while ((len(load_items) > 0) and (avail_time < num_turns)):
        # Distance from drone to warehouse
        dist_warehouse = np.linalg.norm(warehouse_locs - drone_loc_plan, 2, axis = 1)
        
        for warehouse_id in dist_warehouse.argsort():
            # Available product types in this warehouse
            warehouse_avail_prods = np.where(warehouse_prods_plan[warehouse_id] > 0)[0]
            # Remaining product types to load
            target_items = np.array(list(load_items.keys()))
            
            warehouse_load_prodtypes = target_items[np.in1d(target_items, warehouse_avail_prods)]
            
            if len(warehouse_load_prodtypes) > 0:
                
                for prodtype in warehouse_load_prodtypes:
                    
                    # Load minimum of the required number and available number in the warehouse
                    load_num_units = min(load_items[prodtype], warehouse_prods_plan[warehouse_id, prodtype])
                    
                    load_items[prodtype] -= load_num_units
                    if load_items[prodtype] == 0:
                        del load_items[prodtype]
                    
                    warehouse_prods_plan[warehouse_id, prodtype] -= load_num_units
                    
                    avail_time += (np.ceil(dist_warehouse[warehouse_id]) + 1)
                    drone_loc_plan = warehouse_locs[warehouse_id]
                    
                    #drone_solution.append(str(drone_no) + " L " + str(warehouse_id) + " " + str(prodtype) + " " + str(load_num_units))
                    drone_solution.append([drone_no, "L", warehouse_id, prodtype, load_num_units])
                
                break # Break, have to recalculate distance
    
    # Order Delivery Route Planning
    while (len(deliver_items) > 0) and (avail_time < num_turns):
        
        cust_order_no = np.array(list(deliver_items.keys()))
        cust_order_locs = order_locs[cust_order_no]
        
        # Distance from drone to customers
        dist_cust = np.linalg.norm(cust_order_locs - drone_loc_plan, 2, axis = 1)
        
        # Go to nearest order for delivery
        delivery_cust_idx = dist_cust.argmin()
        delivery_order_no = cust_order_no[delivery_cust_idx]
        
        for prodtype, units in deliver_items[delivery_order_no].items():
            
            avail_time += (np.ceil(dist_cust[delivery_cust_idx]) + 1)
            drone_loc_plan = cust_order_locs[delivery_cust_idx]
            
            #drone_solution.append(str(drone_no) + " D " + str(delivery_order_no) + " " + str(prodtype) + " " + str(units))
            drone_solution.append([drone_no, "D", delivery_order_no, prodtype, units])
        
        del deliver_items[delivery_order_no]
    
    # local serach for route planning 
    mr = dr2mr(drone_solution)
    local_search(mr)
        
    drone_solution = mr2dr(mr)
    
    # Update avail_time and drone_loc_plan based on the updated partial solution
    avail_time, drone_loc_plan = recalcDroneLocTime(drone_solution, drone_avail_time[drone_no], 
                                                    drone_curr_loc[drone_no], warehouse_locs, 
                                                    order_locs)
    
    # Convert partial solution to list of strings
    drone_solution = partialSolnStrOutput(drone_solution)
        
    if avail_time <= num_turns:
        #print("soln found")
        solution += drone_solution
        drone_curr_loc[drone_no] = drone_loc_plan
        drone_avail_time[drone_no] = avail_time
        # Persist np arrays
        order_prods = order_prods_plan
        warehouse_prods = warehouse_prods_plan
        # Counter to keep track how many rounds of no solution found
        skip_counter = 0
    else:
        skip_counter += 1

if (np.sum(order_prods) > 0):
    wait_drones = np.where(drone_avail_time < num_turns)[0]
    for wait_drone in wait_drones:
        solution.append(str(wait_drone) + " W " + str(num_turns - drone_avail_time[wait_drone]))

    drone_avail_time[wait_drones] = num_turns

In [ ]:
with open('/kaggle/working/submission.csv', 'w') as fd:
    fd.write(str(len(solution)) + '\n')
    for writeLine in solution:
        fd.write(writeLine + '\n')